In [1]:
import os
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
import time
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
import sys
sys.path.append(r'C:\Users\tacuevas\Documents\GitHub\concorde\modules')
from mda import anti_neighbors_vectorized, fnorm, fnorm_inv, normalize_df, unnormalize_df
from plots import scatter_interpolate
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
plt.rcParams.update({'font.size': 14})
from matplotlib.offsetbox import AnchoredText
from itertools import combinations, permutations

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

#### Check new version of anti neighbors

In [ ]:
# df = pd.read_pickle(r'..\data\STORM\processed\STORM_NA_R2_In_pnt.pkl')
# df.index = range(len(df))
# df = df.drop(['year', 'month', 'yr_tc_number', 'tc_id', 'max_cat'], axis = 1)
# df = df.fillna(0)
# df.head()

In [ ]:
# index = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]

In [ ]:
# columns = ['lat_at_min_distNC', 'lon_at_min_distNC', 'min_press_at_min_press', 'max_ws_at_min_press', 'rad_to_max_ws_at_min_press',
#           'mean_trajectory']
# index = [1, 1, 1, 1, 1, 0]
# df = df.loc[:, columns]

###### WIthout definitive function

In [ ]:
# norm = normalize_df(df, index)

In [ ]:
# norm.head()

In [ ]:
# a, b = [], []
# for i, j in enumerate(index):
#     if j == 0:
#         b.append(i)
#     elif j == 1:
#         a.append(i)

In [ ]:
# for ib, bi in enumerate(b):
#     aux = norm.iloc[:, bi].values
#     m1 = np.broadcast_to(aux, (len(norm), len(norm)))
#     if ib == 0:
#         mdir = np.minimum(np.abs(m1 - m1.T), 2 - np.abs(m1 - m1.T))
#     else:
#         mdir = mdir + np.minimum(np.abs(m1 - m1.T), 2 - np.abs(m1 - m1.T))

# aux = norm.iloc[:, a].values
# msca = cdist(aux, aux, metric = 'sqeuclidean')

# m = msca + mdir
# del msca
# del mdir

In [ ]:
# t0 = time.time()

# k = 1000

# points = [tuple(x) for x in norm.values]
# dummy_index = list(df.index.values)
# remaining_points = points[:]
# solution_set = []
# index_solution_set = []

# tree = cKDTree(norm.values)
# meanvalue = list(norm.mean())
# ix_meanvalue = tree.query(meanvalue)[1]

# solution_set.append(points[ix_meanvalue])
# index_solution_set.append(dummy_index[ix_meanvalue])

# # while len(solution_set) < k:
# while len(solution_set) < k:
#     t0 = time.time()
#     if len(solution_set) == 1:
#         ## iter 1, easy one
#         distances = m[index_solution_set[-1], :]
#         newSolIx = distances.argmax()
#         solution_set.append(points[newSolIx])
#         index_solution_set.append(newSolIx)
#     else:
#         ## iter 2, starts to get complicated
#         distances = np.minimum(distances, m[index_solution_set[-1], :])
#         newSolIx = distances.argmax()
#         solution_set.append(points[newSolIx])
#         index_solution_set.append(newSolIx)
# #     print(f'Iter {time.time() - t0:0.4f}: s')
        
# normclusters = pd.DataFrame(columns = df.columns, data = solution_set)
# clusters_wf = unnormalize_df(df, normclusters, index)
# clusters_wf['index_cluster'] = index_solution_set

In [ ]:
# clusters_wf.index_cluster.nunique()

###### Using function

In [ ]:
# clusters = anti_neighbors_vectorized(df, 1000, index)

In [ ]:
# clusters.iloc[:, :-1].equals(clusters_wf)

#### MDA

In [3]:
df = pd.read_pickle(r'..\data\STORM\processed\STORM_NA_R2_In_pnt.pkl')
df.index = range(len(df))
df = df.drop(['year', 'month', 'yr_tc_number', 'tc_id', 'max_cat'], axis = 1)
# df = df.fillna(0)
df.head()

,time_step_at_min_press,time_step_at_min_distNC,lat_at_min_press,lon_at_min_press,lat_at_min_distNC,lon_at_min_distNC,min_press_at_min_distNC,min_press_at_min_press,max_ws_at_min_distNC,max_ws_at_min_press,...,distNC_at_min_distNC,distNC_at_min_press,trajectory_at_min_distNC,trajectory_at_min_press,mean_trajectory,curvature,duration,lat_gen,lon_gen,length
0,25.0,33.0,29.6,-76.2,30.5,-79.3,977.2,953.7,36.800000,50.8,...,571.303714,623.563360,284.036243,303.690068,306.734075,1.149159e-13,46.0,22.1,-67.9,1895.686058
1,20.0,49.0,17.2,-61.5,33.2,-68.2,954.5,939.2,46.600000,52.7,...,822.784684,2505.942098,21.801409,284.036243,347.184714,5.722621e-14,74.0,13.4,-49.4,6182.285591
2,4.0,8.0,25.9,-83.4,27.4,-82.8,1000.5,993.2,16.500000,22.0,...,1035.895766,1209.183408,36.869898,18.434949,19.242162,0.000000e+00,8.0,24.6,-83.8,331.580145
3,4.0,9.0,25.9,-83.4,27.7,-82.5,1000.5,993.2,16.525852,22.0,...,992.030799,1209.183408,45.000000,18.434949,22.104144,0.000000e+00,9.0,24.6,-83.8,376.112025
4,2.0,3.0,29.1,-76.9,29.2,-77.2,1003.8,1000.4,15.800000,18.5,...,666.410514,676.494047,288.434949,288.434949,276.144983,0.000000e+00,3.0,29.1,-76.6,60.426392


In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
time_step_at_min_press,32247.0,2.040475e+01,2.035621e+01,0.000000e+00,5.000000,14.000000,2.900000e+01,2.220000e+02
time_step_at_min_distNC,32247.0,2.971759e+01,2.836426e+01,0.000000e+00,7.000000,19.000000,4.700000e+01,2.340000e+02
lat_at_min_press,32247.0,2.676754e+01,7.654801e+00,6.900000e+00,21.200000,26.500000,3.150000e+01,5.990000e+01
lon_at_min_press,32247.0,-6.933692e+01,1.067409e+01,-9.890000e+01,-77.100000,-70.500000,-6.270000e+01,-1.200000e+00
lat_at_min_distNC,32247.0,3.041307e+01,5.287352e+00,2.010000e+01,26.400000,31.200000,3.460000e+01,4.580000e+01
lon_at_min_distNC,32247.0,-7.318127e+01,6.704466e+00,-8.490000e+01,-78.700000,-73.800000,-6.770000e+01,-6.010000e+01
min_press_at_min_distNC,32247.0,9.862400e+02,2.011649e+01,9.034000e+02,975.400000,996.100000,1.000900e+03,1.012000e+03
min_press_at_min_press,32247.0,9.657081e+02,2.675121e+01,8.834000e+02,945.700000,971.500000,9.894000e+02,1.004200e+03
max_ws_at_min_distNC,32247.0,2.681077e+01,1.283903e+01,8.488401e+00,17.100000,20.900000,3.520000e+01,7.500000e+01
max_ws_at_min_press,32247.0,3.917857e+01,1.537323e+01,1.720000e+01,25.300000,35.700000,5.180000e+01,8.350287e+01


In [4]:
index = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]

In [5]:
cl0 = anti_neighbors_vectorized(df, 1000, index)

100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [01:02<00:00, 15.94it/s]


In [6]:
print(f'min weight: {cl0.weight.min()*100:0.5f}')
print(f'max weight: {cl0.weight.max()*100:0.5f}')

min weight: 0.00310
max weight: 1.33036


In [7]:
cl1 = anti_neighbors_vectorized(df, 750, index)

100%|████████████████████████████████████████████████████████████████████████████████| 749/749 [00:32<00:00, 22.93it/s]


In [8]:
print(f'min weight: {cl1.weight.min()*100:0.5f}')
print(f'max weight: {cl1.weight.max()*100:0.5f}')

min weight: 0.00310
max weight: 1.56294


In [9]:
cl2 = anti_neighbors_vectorized(df, 500, index)

100%|████████████████████████████████████████████████████████████████████████████████| 499/499 [00:14<00:00, 34.51it/s]


In [10]:
print(f'min weight: {cl2.weight.min()*100:0.5f}')
print(f'max weight: {cl2.weight.max()*100:0.5f}')

min weight: 0.00310
max weight: 1.60325


In [11]:
cl0.to_pickle(r'..\data\STORM\processed\MDA_output\STORM_NA_R2_In_pnt_Clusters_MDAvec_1000_allvars.pkl')
cl1.to_pickle(r'..\data\STORM\processed\MDA_output\STORM_NA_R2_In_pnt_Clusters_MDAvec_0750_allvars.pkl')
cl2.to_pickle(r'..\data\STORM\processed\MDA_output\STORM_NA_R2_In_pnt_Clusters_MDAvec_0500_allvars.pkl')

#### Plot clusters

In [12]:
perm = list(permutations(df.columns, 2))

In [13]:
pathout = Path(r'..\data\STORM\processed\MDA_output\PLOTS_STORM_NA_R2_In_pnt_Clusters_MDAvec_allvars')

In [14]:
for p in tqdm(perm):
    xcol = p[0]
    ycol = p[1]
    
    x = df.loc[:, xcol]
    y = df.loc[:, ycol]
    z = scatter_interpolate(x,y)
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]
    cm = plt.cm.get_cmap('cividis')

    fig, ax = plt.subplots(figsize = (6, 4))
    ax.scatter(x, y, c = z, cmap = cm, alpha = 0.5)
    ax.scatter(cl0.loc[:, xcol], cl0.loc[:, ycol], c = 'r', s = 5)
    ax.scatter(cl1.loc[:, xcol], cl1.loc[:, ycol], c = 'orange', s = 5)
    ax.scatter(cl2.loc[:, xcol], cl2.loc[:, ycol], c = 'magenta', s = 5)
    
    ax.set_xlabel(xcol)
    ax.set_ylabel(ycol)
    
    text = AnchoredText(nb_name, 'upper right',frameon = False, borderpad = -2, prop=dict(fontsize = 'xx-small', alpha = 0.4))
    ax.add_artist(text)
    
    fig.savefig(pathout/f'{xcol}_VS_{ycol}.png', dpi = 100, bbox_inches = 'tight')
    plt.close()

100%|████████████████████████████████████████████████████████████████████████████████| 506/506 [07:06<00:00,  1.19it/s]
